<div>
    <img src="./Bilder/Netz.png" style="float: center;height: 50em;">
</div>

In [ ]:
from __future__ import print_function

import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

import matplotlib.pyplot as plt
#import matplotlib
#matplotlib.use('nbagg')
#%matplotlib widget
%matplotlib inline
import numpy as np

In [ ]:
def get_approximate_num_macs(setup):
    width = 28
    height = 28
    n_channels = 1
    macs = 0
    for conv in setup:
        macs += conv["kernel_size"] * conv["kernel_size"] * width * height * n_channels * conv["num_channels"]

        width = width / conv["max_pooling"]
        height = height / conv["max_pooling"]
        n_channels = conv["num_channels"]

    macs += width * height * n_channels * 10
    return macs

import copy

class Net(nn.Module):
    def __init__(self, setup):
        super(Net, self).__init__()
        # Parameters
        self.intermediate_outputs = {}

        self.setup = setup

        # Input layer
        in_channels = 1
        width = 28
        height = 28

        self.convs = nn.ModuleList()
        for layer_params in setup:
            self.convs.append(nn.Sequential(
                nn.Conv2d(
                    in_channels=in_channels, 
                    out_channels=layer_params["num_channels"],
                    kernel_size=layer_params["kernel_size"],
                    stride=1,
                    padding="same",
                ),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=layer_params["max_pooling"], stride=layer_params["max_pooling"]),
            ))

            in_channels = layer_params["num_channels"]
            width = width // layer_params["max_pooling"]
            height = height // layer_params["max_pooling"]

        # Fully connected layer
        fc_in = in_channels * width * height
        fc_out = 10  # channels
        
        self.fc = nn.Linear(
            in_features=fc_in, out_features=fc_out
        )

    def forward(self, x):
        self.intermediate_outputs['in'] = x.detach().numpy()
        for i, conv in enumerate(self.convs):
            x = conv(x)
            self.intermediate_outputs['conv' + str(i+1)] = x.detach().numpy()
        x = torch.flatten(x, 1)
        x = self.fc(x)
        self.intermediate_outputs['fc'] = x.detach().numpy()
        
        output = F.softmax(x, dim=1)
        self.intermediate_outputs['output'] = output.detach().numpy()

        return output
    
    def reset_all_weights(self):
        """
        refs:
            - https://discuss.pytorch.org/t/how-to-re-set-alll-parameters-in-a-network/20819/6
            - https://stackoverflow.com/questions/63627997/reset-parameters-of-a-neural-network-in-pytorch
            - https://pytorch.org/docs/stable/generated/torch.nn.Module.html
        """

        @torch.no_grad()
        def weight_reset(m: nn.Module):
            # - check if the current module has reset_parameters & if it's callable called it on m
            reset_parameters = getattr(m, "reset_parameters", None)
            if callable(reset_parameters):
                m.reset_parameters()

        # Applies fn recursively to every submodule see: https://pytorch.org/docs/stable/generated/torch.nn.Module.html
        self.apply(fn=weight_reset)


def training_step(data, target, model, optimizer):
    optimizer.zero_grad()
    output = model(data)
    loss = F.cross_entropy(output, target)
    loss.backward()
    optimizer.step()

    model.intermediate_outputs['target'] = target
    return output, loss

def test_step(data, target, model):
    output = model(data)
    test_loss = F.cross_entropy(output, target, reduction='sum').item()  # sum up batch loss
    pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    correct = pred.eq(target.view_as(pred)).sum().item()

    model.intermediate_outputs['target'] = target
    return output, test_loss, correct


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            _, test_loss_step, correct_step = test_step(data, target, model)
            test_loss += test_loss_step
            correct += correct_step

    test_loss /= len(test_loader.dataset)
    test_acc = 100. * correct / len(test_loader.dataset)

    model.intermediate_outputs['test_loss'] = test_loss
    model.intermediate_outputs['test_acc'] = test_acc

    # print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    #     test_loss, correct, len(test_loader.dataset),
    #     test_acc))

    return model.intermediate_outputs


def train_test_step(logging_interval, data, target, model, device, train_loader, test_loader, optimizer, epoch, batch_idx):
    output, loss = training_step(data, target, model, optimizer)
    intermediate_outputs_train = copy.deepcopy(model.intermediate_outputs)

    if batch_idx % logging_interval == 0:
        # print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        #    epoch, batch_idx * len(data), len(train_loader.dataset),
        #    100. * batch_idx / len(train_loader), loss.item()))

        train_loss = loss.item()
        train_acc  = 100. * torch.sum(output.argmax(dim=1) == target).item() / len(target)

        # Test
        intermediate_outputs_test = test(model, device, test_loader)
        model.train()

        intermediate_outputs_train['train_loss'] = train_loss
        intermediate_outputs_train['train_acc'] = train_acc
    else:
        intermediate_outputs_test = None
        
    return intermediate_outputs_train, intermediate_outputs_test

def plot_cost_functions(axs, train_accs, test_accs, train_losses, test_losses):
    
    ax = axs[0]  # Accuracy

    max_visible_points = 100
    xtmp = np.arange(len(train_accs) - min(len(train_accs), max_visible_points), len(train_accs))
    if not ax.lines:  # decorate
        ax.plot(xtmp, train_accs)
        ax.plot(xtmp, test_accs)
        ax.set_xlabel('Batch Number'); ax.set_ylabel('Percentages')
        ax.legend(['Training Accuracy (' + str(train_accs[-1]) + '%)', 'Test Accuracy (' + str(test_accs[-1]) + '% / ' + str(max(test_accs)) + '%)'])
    else:  # update
        ax.lines[0].set_data(xtmp, train_accs[len(train_accs) - min(len(train_accs), max_visible_points):])
        ax.lines[1].set_data(xtmp, test_accs[len(train_accs) - min(len(train_accs), max_visible_points):])
        ax.legend(['Training Accuracy (' + str(train_accs[-1]) + '%)', 'Test Accuracy (' + str(test_accs[-1]) + '% / ' + str(max(test_accs)) + '%)'])
        ax.relim(); ax.autoscale_view(True, True, True)

    print('Training Accuracy (' + str(train_accs[-1]) + '%),\t Test Accuracy (' + str(test_accs[-1]) + '% / Best: ' + str(max(test_accs)) + '%)')
    ax = axs[1]  # Loss

    xtmp = np.arange(len(train_losses))
    if not ax.lines:  # decorate
        ax.plot(train_losses)
        ax.plot(test_losses)
        ax.set_xlabel('Batch Number'); 
        ax.legend(['Training Loss', 'Test Loss'])
    else:  # update
        ax.lines[0].set_data(xtmp, train_losses)
        ax.lines[1].set_data(xtmp, test_losses)
        ax.relim(); ax.autoscale_view(True, True, True)


def plot_test_labels(ax, target, act_fully, act_softmax):

    three_red = np.repeat('r', 3)

    softmax_topthree = sorted(act_softmax)[-3:]
    percentages =  100*np.maximum(softmax_topthree, 0)
    target = int(target)
    idx_topthree = np.argpartition(act_fully, -3)[-3:]
    if not ax.containers:                
        ax.set_xticks([]); ax.set_yticks(np.arange(3)); 
    else:
        ax.containers[0].remove()

    ax.set_yticklabels(idx_topthree)

    colors = three_red.copy()
    colors[idx_topthree == target] = 'g'  # Mark target class in green
    ax.barh(np.arange(3), percentages, align='center', color=colors)
    for m, v in enumerate(percentages):
        ax.text(v + 0.05, m , f"{v:2.2f}%", color='black')

    ax.set_xlim((0, 100))

def plot_image(im, ax, **kwargs):
    if not ax.images:  # Decorate
        ax.imshow(im, cmap='gray', **kwargs)
        ax.set_xticks([]); ax.set_yticks([])
    else:  # Update
        ax.images[0].set_array(im)


def plot_weights(weights, ax, **kwargs):
    (ch_out, ch_in, h, w) = weights.shape

    weights2 = weights.transpose(1, 2, 0, 3).reshape(ch_in*h, ch_out*w)

    if not ax.images:  # Decorate
        ax.imshow(weights2, cmap='gray', **kwargs)
        ax.set_xticks([]); ax.set_yticks([])

        if ch_out > 1:
            for tmp in range(ch_out):
                ax.axvline(tmp*w-0.5, color='black', lw=1)
        if ch_in > 1:
            for tmp in range(ch_in):
                ax.axhline(tmp*h-0.5, color='black', lw=1)
    
    else:  # Update
        ax.images[0].set_array(weights2)


def plot_activations(act, ax, **kwargs):
    (ch_out, h, w) = act.shape

    act2 = act.transpose(1, 0, 2).reshape(h, ch_out*w)
    ch_in = 1
    if ch_out > 15:
        act2 = np.concatenate(np.split(act2, 2, axis=1))
        ch_out = ch_out // 2
        ch_in = ch_in * 2

    if not ax.images:  # Decorate

        ax.imshow(act2, cmap='gray', **kwargs)
        ax.set_xticks([]); ax.set_yticks([]) 

        if ch_out > 1:
            for tmp in range(ch_out):
                ax.axvline(tmp*w-0.5, color='black', lw=1)

        if ch_in > 1:
            for tmp in range(ch_in):
                ax.axhline(tmp*h-0.5, color='black', lw=1)

    else:  # Update
        ax.images[0].set_array(act2)


def plot_activations_figure(axs, input, act1, act2, act3, act4, weights1, weights2):
    plot_image(input, axs[0][0])
    plot_activations(act1, axs[0][1])
    plot_activations(act2, axs[0][2])
    plot_activations(act3[:,np.newaxis,np.newaxis], axs[0][3])
    plot_activations(act4[:,np.newaxis,np.newaxis], axs[0][4])
    plot_image(input, axs[1][0])
    plot_weights(weights1, axs[1][1])
    plot_weights(weights2, axs[1][2])

from IPython.display import clear_output

def optimize_and_plot_model(model, learning_rate, batch_size, num_steps):
    plt.cla()
    plt.ion()
    subplot_shape = (10, 9)

    pos_test = (5, 0)
    axs_test = [
        plt.subplot2grid(subplot_shape, (0 + pos_test[0], 0 + pos_test[1])),
        plt.subplot2grid(subplot_shape, (0 + pos_test[0], 1 + pos_test[1])),
        plt.subplot2grid(subplot_shape, (0 + pos_test[0], 2 + pos_test[1])),
        plt.subplot2grid(subplot_shape, (0 + pos_test[0], 3 + pos_test[1])),
        plt.subplot2grid(subplot_shape, (1 + pos_test[0], 0 + pos_test[1])),
        plt.subplot2grid(subplot_shape, (1 + pos_test[0], 1 + pos_test[1])),
        plt.subplot2grid(subplot_shape, (1 + pos_test[0], 2 + pos_test[1])),
        plt.subplot2grid(subplot_shape, (1 + pos_test[0], 3 + pos_test[1])),
        plt.subplot2grid(subplot_shape, (2 + pos_test[0], 0 + pos_test[1])),
        plt.subplot2grid(subplot_shape, (2 + pos_test[0], 1 + pos_test[1])),
        plt.subplot2grid(subplot_shape, (2 + pos_test[0], 2 + pos_test[1])),
        plt.subplot2grid(subplot_shape, (2 + pos_test[0], 3 + pos_test[1])),
        plt.subplot2grid(subplot_shape, (3 + pos_test[0], 0 + pos_test[1])),
        plt.subplot2grid(subplot_shape, (3 + pos_test[0], 1 + pos_test[1])),
        plt.subplot2grid(subplot_shape, (3 + pos_test[0], 2 + pos_test[1])),
        plt.subplot2grid(subplot_shape, (3 + pos_test[0], 3 + pos_test[1])),
        plt.subplot2grid(subplot_shape, (4 + pos_test[0], 0 + pos_test[1])),
        plt.subplot2grid(subplot_shape, (4 + pos_test[0], 1 + pos_test[1])),
        plt.subplot2grid(subplot_shape, (4 + pos_test[0], 2 + pos_test[1])),
        plt.subplot2grid(subplot_shape, (4 + pos_test[0], 3 + pos_test[1])),
    ]
    axs_test[0].figure.set_size_inches(20, 10)
    axs_test[0].margins(0.1)
    axs = [
        plt.subplot2grid(subplot_shape, (0, 0), colspan=4, rowspan=2),
        plt.subplot2grid(subplot_shape, (2, 0), colspan=4, rowspan=2),
    ] 

    ax_model_params = plt.subplot2grid(subplot_shape, (4, 0), colspan=4, rowspan=1)

    axs_weights = [
        [
            plt.subplot2grid(subplot_shape, (0, 5), rowspan=2, colspan=2),
            plt.subplot2grid(subplot_shape, (3, 5), rowspan=2, colspan=2),
            plt.subplot2grid(subplot_shape, (5, 5), rowspan=2, colspan=2),
            plt.subplot2grid(subplot_shape, (7, 5), rowspan=2, colspan=2),
            plt.subplot2grid(subplot_shape, (9, 5), rowspan=1, colspan=2),
        ],
        [
            plt.subplot2grid(subplot_shape, (0, 7), rowspan=2, colspan=2),
            plt.subplot2grid(subplot_shape, (3, 7), rowspan=2, colspan=2),
            plt.subplot2grid(subplot_shape, (5, 7), rowspan=2, colspan=2),
        ],
    ]
    axs_weights[0][0].set_title("Input Picture")
    axs_weights[0][1].set_title("Output of Layer 1")
    axs_weights[0][2].set_title("Output of Layer 2")
    axs_weights[0][3].set_title("Output of Fully Connected Layer")
    axs_weights[0][4].set_title("Output of Fully Connected Layer (after Softmax)")

    axs_weights[1][0].set_title("Input Picture")
    axs_weights[1][1].set_title("Filters of Layer 1")
    axs_weights[1][2].set_title("Filters of Layer 2")

    plt.tight_layout()

    # Training settings
    test_batch_size = 24
    epochs = num_steps
    random_seed = 1
    logging_interval = 10

    torch.manual_seed(random_seed)

    device = torch.device("cpu")

    train_kwargs = {'batch_size': batch_size}
    test_kwargs = {'batch_size': test_batch_size}

    # Data
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])
    dataset1 = datasets.MNIST('../data', train=True, download=True, transform=transform)
    dataset2 = datasets.MNIST('../data', train=False, transform=transform)
    train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
    test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

    # Model
    model = model.to(device)
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)

    model_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
    model_operations = get_approximate_num_macs(model.setup)

    ax_model_params.text(0.5, 0, "Num parameters: " + str(int(model_parameters)) + "\n" + "Num operations per picture: " + str(int(model_operations)), horizontalalignment="center", verticalalignment="center")
    ax_model_params.get_xaxis().set_visible(False)
    ax_model_params.get_yaxis().set_visible(False)
    ax_model_params.axis("off")
    
    print("Num parameters: " + str(int(model_parameters)) + "\n" + "Num operations per picture: " + str(int(model_operations)))

    # Optimizer
    intermediate_outputs_test = None
    train_losses = []
    train_accs = []
    test_losses = []
    test_accs = []
    for epoch in range(1, epochs + 1):
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            intermediate_outputs_train, intermediate_outputs_test0 = train_test_step(logging_interval, data, target, model, device, train_loader, test_loader, optimizer, epoch, batch_idx)
            if batch_idx % logging_interval == 0:
                intermediate_outputs_test = intermediate_outputs_test0
                train_losses.append(intermediate_outputs_train['train_loss'])
                train_accs.append(intermediate_outputs_train['train_acc'])
                test_losses.append(intermediate_outputs_test['test_loss'])
                test_accs.append(intermediate_outputs_test['test_acc'])

                l = {}
                for idx in range(intermediate_outputs_test['in'].shape[0]):
                    target = intermediate_outputs_test['target'][idx]
                    act4 = intermediate_outputs_test['fc'][idx, :]
                    output = intermediate_outputs_test['output'][idx, :]

                    if target not in l:
                        l[target] = {}
                        l[target]["output"] = output
                        l[target]["act4"] = act4
                    else:
                        l[target] += output
                        l[target]["output"] = output
                        l[target]["act4"] = act4
                    

                for idx in range(10):
                    im1 = intermediate_outputs_test['in'][idx, 0, :, :]
                    act4 = intermediate_outputs_test['fc'][idx, :]
                    output = intermediate_outputs_test['output'][idx, :]
                    target = intermediate_outputs_test['target'][idx]

                    ax_test = axs_test[2*idx]
                    ax_label = axs_test[2*idx + 1]
                    ax_label.clear()

                    plot_image(im1, ax_test)

                    plot_test_labels(ax_label, target, act4, output)

                plot_cost_functions(axs, train_accs, test_accs, train_losses, test_losses)

                idx = 0
                im1 = intermediate_outputs_train['in'][idx, 0, :, :]

                weights1 = model.convs[0][0].weight.data.detach().numpy()
                weights2 = model.convs[1][0].weight.data.detach().numpy()

                act1 = intermediate_outputs_train['conv1'][idx, :, :, :]
                act2 = intermediate_outputs_train['conv2'][idx, :, :, :]
                act3 = intermediate_outputs_train['fc'][idx, :]
                act4 = intermediate_outputs_train['output'][idx, :]
                
                plot_activations_figure(axs_weights, im1, act1, act2, act3, act4, weights1, weights2)
                #clear_output(wait=True)
                
                plt.savefig("current_training_status.png")


        intermediate_outputs_train['train_loss'] = train_losses
        intermediate_outputs_train['train_acc'] = train_accs
        intermediate_outputs_test['test_loss'] = test_losses
        intermediate_outputs_test['test_acc'] = test_accs
        
        #scheduler.step()
        optimizer.step()

### Task 3.1

In dieser Aufgabe wird ein einfaches neuronales Netzwerk zur Klassifikation von Zahlen erstellt und trainier. Die erste Aufgabe ist es, gute Trainingsparameter für das vordefinierte CNN zu finden. Danach wird die Architektur des CNNs angepasst und für verschiedene Anwendungsszenarien optimiert.

Subtasks:
1. Finde eine passende Lernrate und Batch-Größe, sodass das Netzwerk mindestens 80% Genauigkeit auf dem Testset erreicht.
2. Finde eine Netzwerk-Architektur, sodass mindestens 95% Genauigkeit auf dem Testset erreicht wird.
3. Optimiere eine Netzwerkarchitektur mit weniger als 25000 Operationen pro bild, die trotzdem mindestens 85% Genauigkeit auf dem Testset erreicht. Ein solches CNN würde benutzt, wenn die Rechenpower sehr beschränkt ist (z.B. auf einem Mikrocontroller oder Smartphone).
4. Optimiere ein möglichst kleines CNN mit weniger als 2500 Parametern, dass trotzdem mindesten 85% genauigkeit erzielt.

In [ ]:
# In der Setup Variable wird die Architektur des CNNs definiert. Diese kann verwendet werden um die Architektur des Netzwerks anzupassen.
# Eine weiter Ebene kann dem Netzwerk hinzugefügt werden, indem ein weiterer Eintrag der Liste hinzugefügt wird. 
# Dabei müssen die folgenden Parameter definiert sein:
#   - kernel_size: Größe des Filters in einer "convolutional layer" (z.B. eine Filter-Größe von 5 würde zu einem Filter der Größe 5x5 korrespondieren)
#   - num_channels: Anzahl Ausgabe-Kanälen für diese Ebene = Anzahl filter dieser Ebene
#   - max_pooling: Größe des "max-poolings". Z.B. bei einem max-pooling der Größe 2 würde von jedem 2x2 block der größte Wert übertragen
setup = [
    {"kernel_size": 5, "num_channels":  4, "max_pooling": 2},
    {"kernel_size": 3, "num_channels":  8, "max_pooling": 2},
]

model = Net(setup)

model_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
model_operations = get_approximate_num_macs(model.setup)

print("Num parameters: " + str(int(model_parameters)) + "\n" + "Num operations per picture: " + str(int(model_operations)))

In [ ]:
# Die Parameter für das Training werden hier definiert. Falsche oder schlecht gewählte Parameter können in instabilen Training resultieren.
# Eine gute Wahl der Trainingsparameter ist demnach äußerst wichtig
learning_rate = 1
batch_size = 64

# Training and plotting
optimize_and_plot_model(model, learning_rate, batch_size, num_steps=1)

# >>Weiter zum  [__Feedback__](Feedback.ipynb)